# Data In, Data Out

Software is generally useless without the ability to access underlying data stores whether they be in-memory, files, or databases. Data Analysis software maintains this truth.

This exercise focuses primarily on fetching/storing data in various formats. Something which pandas (Panel Datas) + SQLAlchemy + openpyxl stack does efficiently.

## The Trade-off

SQLAlchemy is an **"ORM"** (Object Relational Mapper) which simplifies database connections for almost any source (SQLServer, MySQL, SQLite, and more). However, some of the operations translate to SQL operations which are expensive when compared with other alternatives. For example, it is generally advised when using SQL to avoid using cursor statements and that's exactly what happens at times. **Performance is the cost of convenience.**

## Basic File Input/Output

SQLAlchemy is not necessary for reading or writing to most file types with pandas since its primary use is interacting with databases. Generally, when we work with files it will be one of several types:

- Excel
- SQLite database
- CSV
- pickle
- JSON
- HTML
- XML
- HDF
- etc...

Pandas natively supports each type with respective methods and occasional library dependencies:
- to_excel/read_excel **(with openpyxl)**
- to_sql/read_sql **(with SQLAlchemy)**
- to_csv/read_csv
- to_pickle/read_pickle
- to_json/read_json, etc...

Each method may contain specific options unique to it despite similarities with other read/write methods.

## First Steps

The present methodology and related code may appear overly verbose when introducing new concepts. More common operations, variables, sets, etc... will be refactored to minimize the excessive amount of required (re)typing boilerplate code.

## Our Second Code Block

Our first code block printed "Hello World", but that was a stepping stone. The initial Code Block in many Jupyter notebooks will initialize, load, or otherwise instantiate the environment with:
- required libraries
- required variables
- project constants

We want to be as **"DRY"** (Don't Repeat Yourself) as possible while maintaining that the cells will execute in sequence without errors.

In [ ]:
# First, import any required libraries
import pandas as pd
from pathlib import Path
import shutil

## Reading/Writing CSV Files

The presented code in the following block goes through several steps which produce a copy of the `Person.csv` file named `Person1.csv`.

### Process Overview (Reinforcing Good Practices)

Depending on an given individual's level of experience, there are rituals which we observe. We honor these rituals, not because we want to, but **because we must**. These rituals serve to benefit operations, reinforce accountability, and enhance our ability to audit errors that occur.

As Programmers and Data Analysts, we strive for conventional, pragmatic wisdom. Writing code can be difficult and reviewing code weeks/months/years after it was writen may be nearly impossible without standardization and/or related processes. When possible **be explicit**, not **implicit**. The person most likely to benefit from these practices is *future* you.

### Producing Backups

1. A common first step when working with data files is to archive an unmodified backup of the file/directory. There are many reasons for this, but the most important is to **"CYA"** (Cover Your Ass) **WHEN** an error occurs. (An error or failure **WILL ALWAYS** occur no matter how much talent is present).

2. Similarly, if working with databases, a backup should be created before running any script which attempts to DELETE, UPDATE, or INSERT records.
    - It is otherwise possible to run a script which causes DATA LOSS unintentionally through coding errors
    - **Never, ever, ever test a script on a Production Database**.
        1. Instead, create a backup of the Production Database.
        2. Import the backup into a Development Database.
        3. Then test and inspect results.
        4. Then have others, QA (Quality Assurance) or otherwise, inspect the results.

In [ ]:
# the following heading will be omitted from other code blocks, but it's provided as an example for others to quickly identify the code intent or other important metadata

##
#
#   Script: Produce a copy of a csv in the same data directory
#   Author: CHMBRB
#   Date: 2023-11-22
#   License: BSD-3-Clause
#
##

# create a path object to person csv data (relative to the present notebook)
person_csv = Path("../data/Person.csv")

# read the person csv which uses the tab-delimit character
# NOTE: the low_memory flag option is generally required on Windows Systems for what is arbitrarily considered to be a "large file"
df = pd.read_csv(person_csv, sep="\t", index_col='id', low_memory=False)
# df = pd.read_csv(person_csv, **CONSTANT['csv_kwargs'])

# declare a new relative csv document path
person_csv_1 = Path("../data/Person1.csv")

# # NOTE: we could use the os library to produce a copy of the document without loading it into pandas and there's good reason to consider that approach over the one presented!
# # instead, assume we did operations on the data and want to persist those changes in a new file, ie. The original file is the backup (not advised)
# df.to_csv(person_csv_1, sep='\t')

# NOTE: additional code (written as a function) to perform file copy using shutil.copy to ensure that Pandas doesn't mangle the original file contents
def copy_file(p1, p2):
    """Copy a file using the shutil library for two file system Path objects (p1, p2)"""
    shutil.copy(p1, p2)


# call the previously created function to produce a copy
copy_file(person_csv, person_csv_1)


## Reading/Writing SQLite Databases

SQLite is a cross-platform application for persisting SQLite databases on local disk. It is a great tool for development despite several caveats and one potential advantage of using SQLite includes data manipulation with ANSI SQL (if team members are more comfortable using SQL).

### Overview

In this example, we import the required SQLAlchemy dependency and instantiate a "database engine" using a connection string. Pandas then writes the entries to the specified table with several common method option keyword arguments (`**kwargs`).

In [ ]:
# database connection imports and setup
from sqlalchemy import create_engine

# database connection string (this will vary for each database type and may require additional dependencies, drivers, etc...)
# see also: https://docs.sqlalchemy.org/en/20/core/engines.html
connection_string = "sqlite:///../data/sqlite.db"

# instantiate the database engine
engine = create_engine(connection_string)

# write the dataframe to the 'persons' database table
df.to_sql(name="persons", con=engine, if_exists='replace', index_label='id')

## Process Automation

While we could utilize the approach previously presented to migrate data from multiple files, that may become tedious, acrobatic, and increase the likelihood for introducing errors. Plus, I dislike typing more than I have to unless there's an associated achievement award and there may be many more than just 3 CSV files to import. 

### Overview

Instead of loading each individual CSV file into a DataFrame and then using the SQLAlchemy engine to write the data to a corresponding table, we'll construct a loop which iterates over the necessary files and performs the required migration sequence for each respective database table.

In [ ]:
# define the directory which contains the relevant file set
path_dir = Path("../data/")

# NOTE: this technique is common in python and referred to as a "list comprehension" or "set comprehension"; That's because it can be used to create new [] or {} objects (or other iterables)
# here we collect all files ending with .csv EXCEPT for the one previously created to avoid duplicates
# alternatively, the list comprehension could be a one-liner:
# file_list = [x for x in path_dir.iterdir() if str(x).endswith('.csv') and str(x) != '../data/Person1.csv']
file_list = [
    # produce an iterator variable
    x for x 
    # define the iterable to work on
    in path_dir.iterdir() 
    # select files ending with '.csv' and EXCLUDE the copy produced earlier in this workbook
    if str(x).endswith('.csv') and str(x) != '../data/Person1.csv'
]

# construct a loop, iterate through the files, and import the content to its respective sqlite database table
for f in file_list:
    # construct a pandas dataframe object from the current file
    df = pd.read_csv(f, sep="\t", index_col='id', low_memory=False)
    # strip the tablename from the filename and interpolate the results using f-strings (format string syntax)
    table_name = f"{str(f).split('/')[-1].split('.')[0].lower()}s"
    # write the dataframe to its respective table
    df.to_sql(name=table_name, con=engine, if_exists='replace', index_label='id')

## Options

At this point, if you and your team are more comfortable using SQL it is possible to query the database created from our CSV files using sqlitebrowser. Otherwise, we'll continue in future notebooks to evaluate data using Python + Pandas, explore creating relevant graphs with Matplotlib, and eventually use some of the related data to bind to PDF files (using PyPDF2) or generate tables and other original documents using Jinja + tabulate.

## Reading SQL Statements

Now that each table exists in our database, we can apply more complex SQL statements to pull the exact necessary datasets accordingly for additional analysis.

In [ ]:
# define a relevant SQL statement to pull the necessary related entries
sql_statement = """SELECT *
                    FROM persons p
                    JOIN contactdetails c
                        ON p.id == c.id
                    JOIN labordetails l
                        ON p.id == l.person_id;"""

# execute sql statement against the database engine and read query results into a dataframe object
df = pd.read_sql(sql_statement, con=engine)

## What About *Real* Databases?

The reason that I neglected to address accessing *real* databases is because it's a bit more nuanced. For example, there are typically licensing restrictions and potentially python database drivers which may have various setup requirements.

To my knowledge, there are at least 2 common Microsoft SQL Server database drivers:
- pyodbc
- pymssql

pyodbc is the __[Official Recommendation](https://learn.microsoft.com/en-us/sql/connect/python/python-driver-for-sql-server?view=sql-server-ver16)__.

However, for this section we will make use of Postgres because it offers the most permissive licensing when compared with Microsoft SQL Server and MySQL.

<!-- TODO: Create a docker compose file for this project's required services... -->

<!-- Is this true? https://stackoverflow.com/questions/49888007/understanding-mysql-licensing -->

## Closing Remarks

After having completed this section, users should now be able to successfully import/export data using Python Pandas.